In [1]:
!pip install numpy


In [2]:
!pip install matplotlib


In [3]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 12.5 MB/s eta 0:00:00


In [4]:
!pip install deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.5 MB/s eta 0:00:00


In [5]:
!pip install opencv-python-headless


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2

# Initialize YOLO model
model_path = '/kaggle/input/detection-model-best/best.pt'  # Replace with your model path
model = YOLO(model_path)

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video path
video_path = '/kaggle/input/sheep-video/Sheepeating.mp4'  # Replace with your video path
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video settings
output_video_path = '/kaggle/working/processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Movement data structures
sheep_tracks = defaultdict(list)
prev_positions = {}  # Store previous positions for speed calculation
abnormal_ids = set()  # Track IDs with abnormal speed

# Speed thresholds (pixels/frame)
NORMAL_SPEED_MIN = 10.0
NORMAL_SPEED_MAX = 20.0

# Process video frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or unable to read frame.")
        break

    # Detect sheep using YOLO
    results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
    detections = []
    for result in results:
        for box in result.boxes:
            try:
                x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                if confidence > 0.5 and class_id == 0:  # Sheep class
                    detections.append([[x1, y1, x2, y2], confidence])
            except Exception as e:
                print(f"Error processing box: {e}")
                continue

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        x, y, w, h = track.to_ltwh()
        center_x = int(x + w / 2)
        center_y = int(y + h / 2)

        # Track movement
        sheep_tracks[track_id].append((center_x, center_y))

        # Calculate speed
        if track_id in prev_positions:
            prev_x, prev_y = prev_positions[track_id]
            speed = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)

            # Flag abnormal speeds
            if speed < NORMAL_SPEED_MIN or speed > NORMAL_SPEED_MAX:
                abnormal_ids.add(track_id)
                cv2.putText(frame, f"ID {track_id} (Abnormal Speed)", (int(x), int(y) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        prev_positions[track_id] = (center_x, center_y)

        # Draw bounding box and ID
        color = (0, 0, 255) if track_id in abnormal_ids else (0, 255, 0)
        cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
        cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write frame to output video
    out.write(frame)
    frame_count += 1

# Release resources
cap.release()
out.release()

# Plot sheep movement
plt.figure(figsize=(12, 8))
for track_id, path in sheep_tracks.items():
    x_coords = [p[0] for p in path]
    y_coords = [p[1] for p in path]
    linestyle = '--' if track_id in abnormal_ids else '-'
    color = 'red' if track_id in abnormal_ids else 'green'
    plt.plot(x_coords, y_coords, linestyle=linestyle, color=color, label=f"ID {track_id}")

plt.xlabel("X-Coordinate (pixels)")
plt.ylabel("Y-Coordinate (pixels)")
plt.title("Sheep Movement Paths")
plt.legend()
plt.grid(True)
plt.gca().invert_yaxis()  # Invert Y-axis for video frame coordinates

# Save movement graph
graph_path = '/kaggle/working/sheep_movement_graph.png'
plt.savefig(graph_path)
plt.close()

print("Processing complete.")
print(f"Processed video saved at: {output_video_path}")
print(f"Movement graph saved at: {graph_path}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

0: 480x800 1 sheep, 1164.4ms
Speed: 8.7ms preprocess, 1164.4ms inference, 13.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 sheep, 1008.6ms
Speed: 8.1ms preprocess, 1008.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 sheep, 956.6ms
Speed: 4.0ms preprocess, 956.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 sheep, 980.1ms
Speed: 4.8ms preprocess, 980.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 sheep, 938.7ms
Speed: 5.6ms preprocess, 938.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 1 sheep, 950.6ms
Speed: 4.8ms preproc

In [7]:
import firebase_admin
from firebase_admin import credentials, db
import datetime

# Firebase app name
app_name = 'sheep_monitoring_app'

# Initialize Firebase if not already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)

# Access the initialized app
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase
def update_sheep_data(sheep_id, coordinates, health_status, activity):
    # Create a reference to the Firebase database
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)

    # Format data to be sent to Firebase
    data = {
        "coordinates": coordinates,
        "healthStatus": health_status,
        "activity": activity,
        "timestamp": datetime.datetime.utcnow().isoformat()  # Add a timestamp
    }

    # Update the sheep data in Firebase
    sheep_ref.set(data)
    print(f"Updated data for {sheep_id} in Firebase.")

# Example model data
model_sheep_data = {
    "sheepID1": {
        "coordinates": "34.0522,-118.2437",  # Example coordinates (latitude, longitude)
        "health_status": "healthy",
        "activity": "grazing"
    },
    "sheepID2": {
        "coordinates": "40.7128,-74.0060",
        "health_status": "injured",
        "activity": "resting"
    }
}

# Send data to Firebase for each sheep
for sheep_id, data in model_sheep_data.items():
    update_sheep_data(sheep_id, data["coordinates"], data["health_status"], data["activity"])

ModuleNotFoundError: No module named 'firebase_admin'

In [ ]:
!pip install python-firebase

In [ ]:
!pip install firebase-admin


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime

# Firebase app name
app_name = 'sheep_monitoring_app'

# Initialize Firebase if not already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)

# Access the initialized app
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase
def update_sheep_data(sheep_id, coordinates, speed, status):
    # Create a reference to the Firebase database
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)

    # Format data to be sent to Firebase
    data = {
        "coordinates": coordinates,
        "speed": speed,
        "status": status,
        "timestamp": datetime.datetime.utcnow().isoformat()  # Add a timestamp
    }

    # Update the sheep data in Firebase
    sheep_ref.set(data)
    print(f"Updated data for {sheep_id} in Firebase.")

# Initialize YOLO model
model_path = '/kaggle/input/detection-model-best/best.pt'  # Replace with your model path
model = YOLO(model_path)

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video path
video_path = '/kaggle/input/sheep-video/Sheepeating.mp4'  # Replace with your video path
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video settings
output_video_path = '/kaggle/working/processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Movement data structures
sheep_tracks = defaultdict(list)
prev_positions = {}  # Store previous positions for speed calculation
abnormal_ids = set()  # Track IDs with abnormal speed

# Speed thresholds (pixels/frame)
NORMAL_SPEED_MIN = 10.0
NORMAL_SPEED_MAX = 50.0

# Process video frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or unable to read frame.")
        break

    # Detect sheep using YOLO
    results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
    detections = []
    for result in results:
        for box in result.boxes:
            try:
                x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                if confidence > 0.5 and class_id == 0:  # Sheep class
                    detections.append([[x1, y1, x2, y2], confidence])
            except Exception as e:
                print(f"Error processing box: {e}")
                continue

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        x, y, w, h = track.to_ltwh()
        center_x = int(x + w / 2)
        center_y = int(y + h / 2)

        # Track movement
        sheep_tracks[track_id].append((center_x, center_y))

        # Calculate speed
        speed = 0
        if track_id in prev_positions:
            prev_x, prev_y = prev_positions[track_id]
            speed = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)

            # Flag abnormal speeds
            if speed < NORMAL_SPEED_MIN or speed > NORMAL_SPEED_MAX:
                abnormal_ids.add(track_id)
                cv2.putText(frame, f"ID {track_id} (Abnormal Speed)", (int(x), int(y) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        prev_positions[track_id] = (center_x, center_y)

        # Update Firebase
        status = "abnormal" if track_id in abnormal_ids else "normal"
        update_sheep_data(track_id, f"{center_x},{center_y}", speed, status)

        # Draw bounding box and ID
        color = (0, 0, 255) if track_id in abnormal_ids else (0, 255, 0)
        cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
        cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write frame to output video
    out.write(frame)
    frame_count += 1

# Release resources
cap.release()
out.release()

# Plot sheep movement
plt.figure(figsize=(12, 8))
for track_id, path in sheep_tracks.items():
    x_coords = [p[0] for p in path]
    y_coords = [p[1] for p in path]
    linestyle = '--' if track_id in abnormal_ids else '-'
    color = 'red' if track_id in abnormal_ids else 'green'
    plt.plot(x_coords, y_coords, linestyle=linestyle, color=color, label=f"ID {track_id}")

plt.xlabel("X-Coordinate (pixels)")
plt.ylabel("Y-Coordinate (pixels)")
plt.title("Sheep Movement Paths")
plt.legend()
plt.grid(True)
plt.gca().invert_yaxis()  # Invert Y-axis for video frame coordinates

# Save movement graph
graph_path = '/kaggle/working/sheep_movement_graph.png'
plt.savefig(graph_path)
plt.close()

print("Processing complete.")
print(f"Processed video saved at: {output_video_path}")
print(f"Movement graph saved at: {graph_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime

# Firebase app name
app_name = 'sheep_monitoring_app'

# Initialize Firebase if not already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)

# Access the initialized app
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase
def update_sheep_data(sheep_id, coordinates, speed, status):
    # Create a reference to the Firebase database
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)

    # Format data to be sent to Firebase
    data = {
        "coordinates": coordinates,
        "speed": speed,
        "status": status,
        "timestamp": datetime.datetime.utcnow().isoformat()  # Add a timestamp
    }

    # Update the sheep data in Firebase
    sheep_ref.set(data)
    print(f"Updated data for {sheep_id} in Firebase.")

# Initialize YOLO model
model_path = '/kaggle/input/detection-model-best/best.pt'  # Replace with your model path
model = YOLO(model_path)

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video path
video_path = '/kaggle/input/sheep-video/Sheepeating.mp4'  # Replace with your video path
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video settings
output_video_path = '/kaggle/working/processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Movement data structures
sheep_tracks = defaultdict(list)
prev_positions = {}  # Store previous positions for speed calculation
abnormal_ids = set()  # Track IDs with abnormal speed

# Speed thresholds (pixels/frame)
NORMAL_SPEED_MIN = 100.0
NORMAL_SPEED_MAX = 200.0

# Process video frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or unable to read frame.")
        break

    # Detect sheep using YOLO
    results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
    detections = []
    for result in results:
        for box in result.boxes:
            try:
                x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                confidence = float(box.conf[0])
                class_id = int(box.cls[0])
                if confidence > 0.5 and class_id == 0:  # Sheep class
                    detections.append([[x1, y1, x2, y2], confidence])
            except Exception as e:
                print(f"Error processing box: {e}")
                continue

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        x, y, w, h = track.to_ltwh()
        center_x = int(x + w / 2)
        center_y = int(y + h / 2)

        # Track movement
        sheep_tracks[track_id].append((center_x, center_y))

        # Calculate speed
        speed = 0
        if track_id in prev_positions:
            prev_x, prev_y = prev_positions[track_id]
            speed = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)

            # Flag abnormal speeds
            if speed < NORMAL_SPEED_MIN or speed > NORMAL_SPEED_MAX:
                abnormal_ids.add(track_id)
                cv2.putText(frame, f"ID {track_id} (Abnormal Speed)", (int(x), int(y) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        prev_positions[track_id] = (center_x, center_y)

        # Update Firebase
        status = "abnormal" if track_id in abnormal_ids else "normal"
        update_sheep_data(track_id, f"{center_x},{center_y}", speed, status)

        # Draw bounding box and ID
        color = (0, 0, 255) if track_id in abnormal_ids else (0, 255, 0)
        cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
        cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write frame to output video
    out.write(frame)
    frame_count += 1

# Release resources
cap.release()
out.release()

# Plot sheep movement
plt.figure(figsize=(12, 8))
for track_id, path in sheep_tracks.items():
    x_coords = [p[0] for p in path]
    y_coords = [p[1] for p in path]
    linestyle = '--' if track_id in abnormal_ids else '-'
    color = 'red' if track_id in abnormal_ids else 'green'
    plt.plot(x_coords, y_coords, linestyle=linestyle, color=color, label=f"ID {track_id}")

plt.xlabel("X-Coordinate (pixels)")
plt.ylabel("Y-Coordinate (pixels)")
plt.title("Sheep Movement Paths")
plt.legend()
plt.grid(True)
plt.gca().invert_yaxis()  # Invert Y-axis for video frame coordinates

# Save movement graph
graph_path = '/kaggle/working/sheep_movement_graph.png'
plt.savefig(graph_path)
plt.close()

print(f"Movement graph saved at: {graph_path}")

# Calculate and plot sheep distance vs. time
plt.figure(figsize=(12, 8))

for track_id, path in sheep_tracks.items():
    distances = [0]  # Initialize with 0 distance at the starting point
    timestamps = list(range(len(path)))  # Use frame count as a proxy for time

    # Calculate cumulative distance over time
    for i in range(1, len(path)):
        x1, y1 = path[i - 1]
        x2, y2 = path[i]
        distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        distances.append(distances[-1] + distance)  # Cumulative sum of distances

    # Plot the distance vs. time for the current track
    linestyle = '--' if track_id in abnormal_ids else '-'
    color = 'red' if track_id in abnormal_ids else 'green'
    plt.plot(timestamps, distances, linestyle=linestyle, color=color, label=f"ID {track_id}")

plt.xlabel("Time (frames)")
plt.ylabel("Cumulative Distance (pixels)")
plt.title("Sheep Distance vs. Time")
plt.legend()
plt.grid(True)

# Save distance vs. time graph
distance_graph_path = '/kaggle/working/sheep_distance_vs_time_graph.png'
plt.savefig(distance_graph_path)
plt.close()

print(f"Distance vs. Time graph saved at: {distance_graph_path}")
print(f"Processed video saved at: {output_video_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os

# Firebase setup
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")  # Ensure the credentials file is in the same directory
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase
def update_sheep_data(sheep_id, coordinates, speed, health_status):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "speed": speed,
        "health_status": health_status,
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.set(data)

# Initialize YOLO and DeepSORT
model_path = '/kaggle/input/detection-model-best/best.pt'  # Place the model file in the same directory
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video settings
video_path = '/kaggle/input/sheep-video/Sheepeating.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Speed', 'Health Status', 'Timestamp'])

    # Tracking setup
    sheep_tracks = defaultdict(list)
    prev_positions = {}
    abnormal_ids = set()
    NORMAL_SPEED_MIN = 0
    NORMAL_SPEED_MAX = 1.5

    # Process video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detect sheep
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:  # Sheep class
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        # Update tracker
        tracks = tracker.update_tracks(detections, frame=frame)
        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            # Calculate speed
            speed = 0
            if track_id in prev_positions:
                prev_x, prev_y = prev_positions[track_id]
                speed = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)

                # Check for abnormal speeds
                if speed < NORMAL_SPEED_MIN or speed > NORMAL_SPEED_MAX:
                    abnormal_ids.add(track_id)

            prev_positions[track_id] = (center_x, center_y)
            health_status = "abnormal" if track_id in abnormal_ids else "normal"

            # Write to CSV
            writer.writerow([track_id, center_x, center_y, speed, health_status, datetime.datetime.utcnow().isoformat()])

            # Update Firebase
            update_sheep_data(track_id, f"{center_x},{center_y}", speed, health_status)

            # Draw bounding box
            color = (0, 0, 255) if track_id in abnormal_ids else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

# Release resources
cap.release()
out.release()
print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os
from scipy.spatial import ConvexHull

# Firebase setup
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")  # Ensure the credentials file is in the same directory
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

# Function to update sheep data in Firebase
def update_sheep_data(sheep_id, coordinates, speed, health_status, avg_distance, area, stop_count, distance_from_group):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "speed": speed,
        "health_status": health_status,
        "avg_distance": avg_distance,
        "area": area,
        "stop_count": stop_count,
        "distance_from_group": distance_from_group,
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.set(data)

# Initialize YOLO and DeepSORT
model_path = '/kaggle/input/detection-model-best/best.pt'  # Place the model file in the same directory
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video settings
video_path = '/kaggle/input/sheep-video/Sheepeating.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Speed', 'Health Status', 'Timestamp'])

    # Tracking setup
    sheep_tracks = defaultdict(list)
    prev_positions = {}
    abnormal_ids = set()
    NORMAL_SPEED_MIN = 0  # Grazing sheep can stay stationary
    NORMAL_SPEED_MAX = 1.5  # Maximum grazing speed
    grazing_time = defaultdict(int)  # To track grazing-like behavior duration

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detect sheep
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:  # Sheep class
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        # Update tracker
        tracks = tracker.update_tracks(detections, frame=frame)
        group_centroids = []  # For calculating group behavior
        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            # Track movement
            sheep_tracks[track_id].append((center_x, center_y))
            group_centroids.append((center_x, center_y))

            # Calculate speed
            speed = 0
            if track_id in prev_positions:
                prev_x, prev_y = prev_positions[track_id]
                speed = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)

            prev_positions[track_id] = (center_x, center_y)

            # Check movement patterns over time
            trajectory = sheep_tracks[track_id]
            distances = []
            if len(trajectory) > 10:  # Use the last 10 movements
                distances = [
                    np.sqrt((trajectory[i][0] - trajectory[i - 1][0]) ** 2 +
                            (trajectory[i][1] - trajectory[i - 1][1]) ** 2)
                    for i in range(1, len(trajectory))
                ]
                avg_distance = sum(distances) / len(distances)
            else:
                avg_distance = 0

            # Check area covered by movement
            if len(trajectory) > 3:
                points = np.array(trajectory)
                hull = ConvexHull(points)
                area = hull.volume  # Approximate area covered
            else:
                area = 0

            # Check frequency of stops
            stop_count = sum(1 for d in distances if d < 0.5) if len(trajectory) > 1 else 0

            # Group behavior: calculate deviation from group centroid
            if group_centroids:
                group_centroid_x = np.mean([pos[0] for pos in group_centroids])
                group_centroid_y = np.mean([pos[1] for pos in group_centroids])
                distance_from_group = np.sqrt((center_x - group_centroid_x) ** 2 + (center_y - group_centroid_y) ** 2)
            else:
                distance_from_group = 0

            # Combine multiple indicators
            grazing_score = 0
            if avg_distance < 2:  # Small average movement
                grazing_score += 1
            if area < 100:  # Small area indicates grazing
                grazing_score += 1
            if stop_count > 5:  # Frequent stops indicate grazing
                grazing_score += 1
            if distance_from_group < 50:  # Grazing sheep stay near the group
                grazing_score += 1

            # Determine health status
            health_status = "normal" if grazing_score >= 3 else "abnormal"

            # Write to CSV
            writer.writerow([track_id, center_x, center_y, speed, health_status, datetime.datetime.utcnow().isoformat()])

            # Update Firebase
            update_sheep_data(track_id, f"{center_x},{center_y}", speed, health_status, avg_distance, area, stop_count, distance_from_group)

            # Draw bounding box
            color = (0, 0, 255) if health_status == "abnormal" else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

# Release resources
cap.release()
out.release()
print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, deque
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import firebase_admin
from firebase_admin import credentials, db
import datetime
import csv
import os
from scipy.spatial import ConvexHull
from sklearn.cluster import DBSCAN

# Firebase setup
app_name = 'sheep_monitoring_app'
if not firebase_admin._apps:
    cred = credentials.Certificate("/kaggle/input/firebase-credentials-file/credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://sheep-monitoring-7dc33-default-rtdb.firebaseio.com'
    }, name=app_name)
app = firebase_admin.get_app(name=app_name)

def update_sheep_data(sheep_id, coordinates, speed, health_status, avg_distance, area, stop_count, distance_from_group):
    sheep_ref = db.reference(f'sheep/{sheep_id}', app=app)
    data = {
        "coordinates": coordinates,
        "speed": speed,
        "health_status": health_status,
        "avg_distance": avg_distance,
        "area": area,
        "stop_count": stop_count,
        "distance_from_group": distance_from_group,
        "timestamp": datetime.datetime.utcnow().isoformat()
    }
    sheep_ref.set(data)

# Initialize YOLO and DeepSORT
model_path = '/kaggle/input/detection-model-best/best.pt'
model = YOLO(model_path)
tracker = DeepSort(max_age=30, n_init=3, max_iou_distance=0.7)

# Video settings
video_path = '/kaggle/input/sheep-video/Sheepeating.mp4'
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_video_path = 'processed_sheep_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# CSV file setup
csv_path = 'tracking_results.csv'
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sheep ID', 'X Coordinate', 'Y Coordinate', 'Speed', 'Health Status', 'Timestamp'])

    sheep_tracks = defaultdict(list)
    prev_positions = {}
    speed_history = defaultdict(lambda: deque(maxlen=30))
    grazing_history = defaultdict(lambda: deque(maxlen=30))
    group_deviation_history = defaultdict(lambda: deque(maxlen=30))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        # Detect sheep
        results = model.predict(frame, stream=True, conf=0.5, iou=0.45)
        detections = []
        for result in results:
            for box in result.boxes:
                try:
                    x1, y1, x2, y2 = int(box.xyxy[0][0]), int(box.xyxy[0][1]), int(box.xyxy[0][2]), int(box.xyxy[0][3])
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    if confidence > 0.5 and class_id == 0:
                        detections.append([[x1, y1, x2, y2], confidence])
                except Exception as e:
                    print(f"Error processing box: {e}")
                    continue

        # Update tracker
        tracks = tracker.update_tracks(detections, frame=frame)
        group_centroids = []  # For calculating group behavior
        all_positions = []

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x, y, w, h = track.to_ltwh()
            center_x = int(x + w / 2)
            center_y = int(y + h / 2)

            sheep_tracks[track_id].append((center_x, center_y))
            group_centroids.append((center_x, center_y))
            all_positions.append([center_x, center_y])

            # Calculate speed
            speed = 0
            if track_id in prev_positions:
                prev_x, prev_y = prev_positions[track_id]
                speed = np.sqrt((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2)

            prev_positions[track_id] = (center_x, center_y)

            # Check historical trends
            speed_history[track_id].append(speed)
            avg_speed = np.mean(speed_history[track_id])

            trajectory = sheep_tracks[track_id]
            distances = [
                np.sqrt((trajectory[i][0] - trajectory[i - 1][0]) ** 2 +
                        (trajectory[i][1] - trajectory[i - 1][1]) ** 2)
                for i in range(1, len(trajectory))
            ] if len(trajectory) > 1 else []

            avg_distance = sum(distances) / len(distances) if distances else 0
            area = ConvexHull(np.array(trajectory)).volume if len(trajectory) > 3 else 0
            stop_count = sum(1 for d in distances if d < 0.5)

            # Clustering using DBSCAN
            if len(all_positions) > 1:
                clustering = DBSCAN(eps=50, min_samples=2).fit(all_positions)
                cluster_label = clustering.labels_[group_centroids.index((center_x, center_y))]
                distance_from_group = 0 if cluster_label == -1 else np.linalg.norm(np.array([center_x, center_y]) - np.mean(np.array(all_positions), axis=0))
            else:
                distance_from_group = 0

            group_deviation_history[track_id].append(distance_from_group)
            avg_group_deviation = np.mean(group_deviation_history[track_id])

            grazing_score = 0
            if avg_distance < 2: grazing_score += 1
            if area < 100: grazing_score += 1
            if stop_count > 5: grazing_score += 1
            if distance_from_group < 50: grazing_score += 1
            grazing_history[track_id].append(grazing_score)

            # Determine health status
            health_status = "normal" if grazing_score >= 3 else "abnormal"

            # Write to CSV
            writer.writerow([track_id, center_x, center_y, speed, health_status, datetime.datetime.utcnow().isoformat()])

            # Update Firebase
            update_sheep_data(track_id, f"{center_x},{center_y}", speed, health_status, avg_distance, area, stop_count, distance_from_group)

            # Visualization
            color = (0, 0, 255) if health_status == "abnormal" else (0, 255, 0)
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.putText(frame, f"Health: {health_status}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.putText(frame, f"Grazing: {grazing_score}", (int(x), int(y) + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Add legend
        cv2.rectangle(frame, (10, 10), (300, 100), (255, 255, 255), -1)
        cv2.putText(frame, "Legend:", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(frame, "Green: Normal", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.putText(frame, "Red: Abnormal", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        out.write(frame)

cap.release()
out.release()
print(f"Processed video saved at: {output_video_path}")
print(f"Tracking data saved at: {csv_path}")

In [ ]:
!pip install torch


In [ ]:
import torch

# Save the DeepSORT tracker state
tracker_state_path = 'deep_sort_tracker_state.pth'
torch.save(tracker.state_dict(), tracker_state_path)


In [ ]:
# Save the DeepSORT tracker state
tracker_state_path = 'deep_sort_tracker_state.pth'
torch.save(tracker.state_dict(), tracker_state_path)
